In [1]:
using LinearAlgebra, Random
const N = 2

2

In [2]:
#base gates
braket0 = [1 0; 0 0]
braket1 = [0 0; 0 1]
Hm = [1/sqrt(2) 1/sqrt(2); 1/sqrt(2) -1/sqrt(2)]
Xm = [0 1; 1 0]
Ym = [0 -im; im 0]
Zm = [1 0; 0 -1]
Sm = [1 0; 0 im]
Im = [1 0; 0 1]
Vm = [(1+1im)/2 (1-1im)/2; (1-1im)/2 (1+1im)/2]
Um(theta) = [cos(theta) -sin(theta); sin(theta) cos(theta)]
Pm(theta) = [0 0; 0 exp(im*theta)];

In [3]:
Identity(n) = Matrix(I, n, n)

expand(gate, index) = kron(Identity(1 << (index - 1)), gate, Identity(1 << (N - index)))

function control_gate(gate, control, target)
    control = control - 1
    target = target - 1
    if control == target
        return Identity(1 << N)
    elseif control < target
        return kron(Identity(1 << control),
                    kron(braket0, Identity(1 << (target - control)))
                    + kron(braket1, Identity(1 << (target - control - 1)), gate),
                    Identity(1 << (N - target - 1)))
    elseif control > target
        return kron(Identity(1 << target),
                    kron(Identity(1 << (control - target)), braket0)
                    + kron(gate, Identity(1 << (control - target - 1)), braket1),
                    Identity(1 << (N - control - 1)))
    end
end;

In [4]:
#gates
H(n) = expand(Hm, n)
X(n) = expand(Xm, n)
Y(n) = expand(Ym, n)
Z(n) = expand(Zm, n)
S(n) = expand(Sm, n)
XX(n) = expand(Im, n)
U(n, theta) = expand(Um(theta), n)
P(n, theta) = expand(Pm(theta), n)
CNOT(a, b) = control_gate(Xm, a, b)
CZ(a, b) = control_gate(Zm, a, b)
SWAP(a, b) = CNOT(a,b) * CNOT(b,a) * CNOT(a,b)
CCNOT(a, b, t) = real.(control_gate(Vm, b, t) * CNOT(a, b) * adjoint(control_gate(Vm, b, t)) * CNOT(a, b) * control_gate(Vm, a, t))
HN() = kron(fill(Hm, N)...);

In [5]:
single_gates = [:(H) :(X) :(Z)]
duo_gates = [:(CNOT) :(CZ)]

function get_gate()
    if rand() < 0.75
        return Expr(:call, single_gates[rand((1:3))], rand((1:N)))
    else
        return Expr(:call, duo_gates[rand((1:2))], rand((1:N)), rand((1:N)))
    end
end

get_gate (generic function with 1 method)

In [6]:
function full_mutation(program)
    clone = deepcopy(program)
    clone[rand((1:program_size))] = get_gate()
    return clone
end

function index_mutation(program)
    clone = deepcopy(program)
    rnd = rand((1:program_size))
    if clone[rnd].args[1] in single_gates
        clone[rnd].args[2] = rand((1:N))
    elseif clone[rnd].args[1] in duo_gates
        clone[rnd].args[rand((2:3))] = rand((1:N))
    end
    return clone
end

function gate_mutation(program)
    clone = deepcopy(program)
    rnd = rand((1:program_size))
    if clone[rnd].args[1] in single_gates
        clone[rnd].args[1] = single_gates[rand((1:3))]
    elseif clone[rnd].args[1] in duo_gates
        clone[rnd].args[1] = duo_gates[rand((1:2))]
    end
    return clone
end

function permute_mutation(program)
    clone = deepcopy(program)
    rnd = rand((1:program_size), 1, 2)
    temp = clone[rnd[1]]
    clone[rnd[1]] = clone[rnd[2]]
    clone[rnd[2]] = temp
    return clone
end

permute_mutation (generic function with 1 method)

In [7]:
function one_point_crossover(program1, program2)
    clone1 = deepcopy(program1)
    clone2 = deepcopy(program2)
    rnd = rand((1:program_size))
    temp = clone1[rnd:program_size]
    clone1[rnd:program_size] = clone2[rnd:program_size]
    clone2[rnd:program_size] = temp
    return clone1, clone2
end

function two_point_crossover(program1, program2)
    clone1 = deepcopy(program1)
    clone2 = deepcopy(program2)
    rnd = sort(rand((1:program_size),2))
    temp = clone1[rnd[1]:rnd[2]]
    clone1[rnd[1]:rnd[2]] = clone2[rnd[1]:rnd[2]]
    clone2[rnd[1]:rnd[2]] = temp
    return clone1, clone2
end

function uniform_crossover(program1, program2)
    clone1 = deepcopy(program1)
    clone2 = deepcopy(program2)
    for i in 1:program_size
        if clone1[i] == :(oracle(w)) || clone2[i] == :(oracle(w))
            continue
        elseif rand() > 0.5
            temp = clone1[i]
            clone1[i] = clone2[i]
            clone2[i] = temp
        end
    end
    return clone1, clone2
end

uniform_crossover (generic function with 1 method)

In [8]:
ys = []
for i in 1:(1 << N)
    y = zeros(Int8, 1 << N)
    y[i] = 1
    push!(ys, y)
end

bell_state = zeros(Int8, 1 << N)
bell_state[1] = 1
#bell_state = HN() * bell_state

function oracle(w)
    x = ones(1 << N)
    x[w] = -1
    return Diagonal(x)
  end

function run_program(program, w)
    output = bell_state
    for i in program
        if i == :(oracle(w))
            output = oracle(w) * output
        else
            output = eval(i) * output
        end
    end
    return output.^2
end

function fitness(program)
    fitness = 0
    for w in 1:(1 << N)
        output = bell_state
        for i in program
            if i == :(oracle(w))
                output = oracle(w) * output
            else
                output = eval(i) * output
            end
        end
        fitness += sum(map((x,y) -> abs(x-y), ys[w], output.^2))
    end
    return fitness
end

fitness (generic function with 1 method)

In [9]:
H(1)*H(2)*bell_state

4-element Vector{Float64}:
 0.4999999999999999
 0.4999999999999999
 0.4999999999999999
 0.4999999999999999

In [10]:
H(1)*H(1)

4×4 Matrix{Float64}:
  1.0           0.0          -2.23711e-17   0.0
  0.0           1.0           0.0          -2.23711e-17
 -2.23711e-17   0.0           1.0           0.0
  0.0          -2.23711e-17   0.0           1.0

In [11]:
program_size = 12
population_size = 100
nr_epochs = 30

generate_program() = [get_gate() for i in 1:program_size]
population = []
for i in 1:population_size
    temp = generate_program()
    for i in 1:floor(pi/4*sqrt(1 << N))
        push!(temp, :(oracle(w)))
    end
    push!(population, shuffle(temp))
end

In [12]:
mutations = [full_mutation index_mutation gate_mutation permute_mutation]
crossovers = [one_point_crossover two_point_crossover uniform_crossover];

In [13]:
function go(population)
    for i in 1:nr_epochs
        display(i)
        display(fitness(population[1]))
        for i in 1:population_size/2
            parents = sort(shuffle(population)[1:5], by = x -> fitness(x))[1:2]
            childs = uniform_crossover(parents[1], parents[2])
            if rand() < 0.75
                childs = mutations[rand((1:4))].(childs)
            end
            push!(population, childs[1])
            push!(population, childs[2])
        end
        population = sort(population, by = x -> fitness(x))[1:population_size]

        if fitness(population[1]) < 0.1
            display(i)
            return population[1]
        end
    end
    return population[1]
end

go (generic function with 1 method)

In [14]:
display(population[1])
display(index_mutation(population[1]))

13-element Vector{Expr}:
 :(X(1))
 :(Z(2))
 :(CNOT(2, 1))
 :(X(1))
 :(CNOT(2, 1))
 :(H(1))
 :(H(2))
 :(X(2))
 :(X(2))
 :(X(2))
 :(X(2))
 :(oracle(w))
 :(H(1))

13-element Vector{Expr}:
 :(X(1))
 :(Z(2))
 :(CNOT(2, 1))
 :(X(1))
 :(CNOT(2, 1))
 :(H(1))
 :(H(2))
 :(X(2))
 :(X(2))
 :(X(1))
 :(X(2))
 :(oracle(w))
 :(H(1))